In [1]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git
!pip install --quiet accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
import torch

In [3]:
#!pip install session_info
import session_info
import pandas as pd

def generate_requirements_file():

    res = session_info.show()
    res = res.data
    res = res.split("-----")[1]
    res_final = ""
    for i,j in zip(res.split()[1::2],res.split()[0::2]):
        if not pd.isna(j):
            req_str_i = j+"=="+i
            res_final+=req_str_i
            res_final+="\n"

    print(res_final)

generate_requirements_file()

pandas==1.5.3
session_info==1.0.0
torch==2.1.0+cu121
transformers==4.38.0.dev0



In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('failed to connect to gpu')
else:
  print(gpu_info)

Sun Jan 28 13:54:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
llm_model_name = "bigscience/bloom-3b"
text = "which is the nearest planet to sun?"

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [6]:
model_8bit_quant = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [15]:
# response from 8bit quant model
generate_from_model(model_8bit_quant, tokenizer)

'which is the nearest planet to sun?"\nThe answer is that the Earth is the nearest planet'

In [16]:
# response from original model
model_without_quant = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", torch_dtype="auto")
generate_from_model(model_without_quant, tokenizer)

'which is the nearest planet to sun?"\nThe answer is that the Earth is the nearest planet'

In [20]:
mem_fp_wq = model_without_quant.get_memory_footprint()/(1024*1024*1024) # float 16
mem_fp_q = model_8bit_quant.get_memory_footprint()/(1024*1024*1024) # float 8

print("memory footprint of NOT quantised and quantised are : ",round(mem_fp_wq,2),"GB",round(mem_fp_q,2),"GB")

memory footprint of NOT quantised and quantised are :  5.59 GB 3.4 GB


In [21]:
model_4bit_quant = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", load_in_4bit=True)

In [23]:
print("memory footpriint of 4 bit model : ",round(model_4bit_quant.get_memory_footprint()/(1024*1024*1024),2)," GB")

memory footpriint of 4 bit model :  2.3  GB
